<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zadanie 6


1. Model predykcyjny oparty o K-NN, który ma oszacować prawdopodobieństwo zakupu (Purchase=1) dla każdego klienta. Na wejściu biorę wszystkie sensowne cechy z pliku Tayko, oraz wyrzucam trzy kolumny: Purchase (target), sequence_number (identyfikator) oraz Spending (jest kopią targetu —więc jego użycie byłoby klasycznym data leakage).

2. podział danych na train/test 70/30 (losowo, z ziarnem i stratyfikacją po Purchase, żeby zachować balans klas). Ponieważ K-NN liczy odległości, a więc jest wrażliwy na skale zmiennych, standaryzuję cechy liczbowe — fit tylko na train, potem transform na test. Następnie dobieram parametry K-NN na train przez CV: sprawdzam małą siatkę k oraz weights i wybieram konfigurację z najlepszym średnim ROC-AUC.
Finalny model uczony na całym train i na teście wyznaczam proba = Purchase=1 jako ranking klientów. Skuteczność rankingu pokazuję później na  Lift i Cumulative Gains, bo w tym zastosowaniu ważniejsze jest „kogo brać w pierwszej kolejności”, niż sama klasyfikacja przy progu 0.5.

In [24]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [25]:
path = "sample_data/Tayko.csv"
df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   sequence_number       2000 non-null   int64
 1   US                    2000 non-null   int64
 2   source_a              2000 non-null   int64
 3   source_c              2000 non-null   int64
 4   source_b              2000 non-null   int64
 5   source_d              2000 non-null   int64
 6   source_e              2000 non-null   int64
 7   source_m              2000 non-null   int64
 8   source_o              2000 non-null   int64
 9   source_h              2000 non-null   int64
 10  source_r              2000 non-null   int64
 11  source_s              2000 non-null   int64
 12  source_t              2000 non-null   int64
 13  source_u              2000 non-null   int64
 14  source_p              2000 non-null   int64
 15  source_x              2000 non-null   int64
 16  source

In [26]:
df.head()

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,128
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0


In [28]:
# check + rozkład targetu Purchase
shape = df.shape  # (N, p)
dtypes = df.dtypes.value_counts().to_dict()  # typy kolumn
na_total = int(df.isna().sum().sum())  # braki ogółem
na_by_col = df.isna().sum().sort_values(ascending=False)  # braki per kolumna
dup_rows = int(df.duplicated().sum())  # duplikaty wierszy

target = "Purchase"  # target
purchase_dtype = df[target].dtype  # typ targetu
purchase_counts = df[target].value_counts(dropna=False).sort_index()  # liczności
purchase_pct = (df[target].value_counts(normalize=True, dropna=False).sort_index() * 100).round(2)  # %
baseline_rr = float(df[target].mean())  # odsetek klasy 1 (baseline)

print("\nSANITY-CHECK  ===")
print(f"Shape: {shape} | Duplikaty wierszy: {dup_rows} | Braki ogółem: {na_total}")
print(f"Typy kolumn: {dtypes}")
print("\nBraki per kolumna (top 10):")
print(na_by_col.head(10).to_string())

print("\nTARGET: Purchase ===")
print(f"Typ: {purchase_dtype} | Unikalne: {sorted(df[target].dropna().unique().tolist())}")
print(pd.DataFrame({"count": purchase_counts, "pct": purchase_pct}).to_string())
print(f"Baseline (mean(Purchase)=P(Purchase=1)): {baseline_rr:.4f}")



SANITY-CHECK  ===
Shape: (2000, 25) | Duplikaty wierszy: 0 | Braki ogółem: 0
Typy kolumn: {dtype('int64'): 25}

Braki per kolumna (top 10):
sequence_number    0
US                 0
source_a           0
source_c           0
source_b           0
source_d           0
source_e           0
source_m           0
source_o           0
source_h           0

TARGET: Purchase ===
Typ: int64 | Unikalne: [0, 1]
          count   pct
Purchase             
0          1000  50.0
1          1000  50.0
Baseline (mean(Purchase)=P(Purchase=1)): 0.5000


In [29]:
# walidacja leakage: Spending>0 <==> Purchase=1 + decyzja (Spending OUT
#zastosowałem kod z zad. 5 -> ten sam dataset
has_spending = "Spending" in df.columns  # czy jest Spending
has_purchase = "Purchase" in df.columns  # czy jest Purchase

if has_spending and has_purchase:
    chk = df.loc[df["Spending"].notna() & df["Purchase"].notna(), ["Spending", "Purchase"]].copy()  # kompletne
    chk["sp_gt0"] = (chk["Spending"] > 0).astype(int)  # Spending>0 -> 1/0
    chk["purchase01"] = chk["Purchase"].astype(int)  # Purchase -> int

    ct = pd.crosstab(chk["sp_gt0"], chk["purchase01"], dropna=False)  # tabela zgodności
    mismatch = int((chk["sp_gt0"] != chk["purchase01"]).sum())  # ile nie pasuje
    mismatch_rate = (mismatch / len(chk)) if len(chk) else np.nan  # % niezgodności

    print("\nLEAKAGE CHECK: Spending vs Purchase ===")
    print(f"Czy są kolumny? Spending={has_spending} | Purchase={has_purchase}")
    print(f"Liczba porównań (kompletne wiersze): {len(chk)}")
    print("\nTabela zgodności (wiersze: Spending>0, kolumny: Purchase):")
    print(ct.to_string())
    print(f"\nNiezgodności: {mismatch} |  błedy %: {mismatch_rate:.4%}")

    if mismatch_rate <= 0.01:  # 1% jako twardy próg na 'praktycznie reguła'
        print("\nDecyzja: Spending USUWAMY z predyktorów (silny leakage).")
    else:
        print("\nDecyzja: Spending nie jest prostą kopią targetu (brak oczywistego leakage).")

else:
    print("\n=== LEAKAGE CHECK: pominięty ===")
    print(f"Brak wymaganych kolumn: Spending={has_spending} | Purchase={has_purchase}")



LEAKAGE CHECK: Spending vs Purchase ===
Czy są kolumny? Spending=True | Purchase=True
Liczba porównań (kompletne wiersze): 2000

Tabela zgodności (wiersze: Spending>0, kolumny: Purchase):
purchase01    0     1
sp_gt0               
0           999     0
1             1  1000

Niezgodności: 1 |  błedy %: 0.0500%

Decyzja: Spending USUWAMY z predyktorów (silny leakage).


In [32]:
# split na train/test + przygotowanie X/y pod K-NN + scaller

y = df["Purchase"].astype(int)  # target (0/1)
X = df.drop(columns=["Purchase", "Spending", "sequence_number"])  # X bez leakage i ID - zdropowane

X_train, X_test, y_train, y_test = train_test_split(  # split
    X, y, test_size=0.3, random_state=42, stratify=y
)

bin_cols = [c for c in X_train.columns if set(X_train[c].unique()).issubset({0, 1})]  # binarne 0/1
num_cols = [c for c in X_train.columns if c not in bin_cols]                           # liczbowe (np. Freq, *_days_ago)

scaler = StandardScaler()                                # scaler
X_train_num = scaler.fit_transform(X_train[num_cols])    # fit tylko na train
X_test_num  = scaler.transform(X_test[num_cols])         # transform na test tym samym scalerem

X_train_knn = X_train.copy()                             # finalne macierze pod KNN
X_test_knn  = X_test.copy()

X_train_knn[num_cols] = X_train_num                      # podmiana przeskalowanych kolumn
X_test_knn[num_cols]  = X_test_num

# sanity
print("X_train_knn:", X_train_knn.shape, "| X_test_knn:", X_test_knn.shape, "| y_train:", y_train.shape, "| y_test:", y_test.shape)
print("num_cols:", num_cols)


X_train_knn: (1400, 22) | X_test_knn: (600, 22) | y_train: (1400,) | y_test: (600,)
num_cols: ['Freq', 'last_update_days_ago', '1st_update_days_ago']


In [34]:
#dobór k (mała siatka) + opcja weights + walidacja na train (CV) + final fit + proba na teście
k_grid = [3, 5, 7, 9, 11, 15, 21]                     # mała siatka k
w_grid = ["uniform", "distance"]                      # wagi

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)          # stratified CV
best = {"auc": -1.0, "k": None, "w": None}                               # best config

for k in k_grid:                                                         # po k
    for w in w_grid:                                                     # po weights
        m = KNeighborsClassifier(n_neighbors=k, weights=w)               # model
        auc_cv = cross_val_score(m, X_train_knn, y_train, cv=cv, scoring="roc_auc").mean()  # mean AUC(CV)
        if auc_cv > best["auc"]:                                          # update best
            best = {"auc": float(auc_cv), "k": int(k), "w": str(w)}

knn = KNeighborsClassifier(n_neighbors=best["k"], weights=best["w"])     # final model
knn.fit(X_train_knn, y_train)                                            # fit na całym train

proba_test = knn.predict_proba(X_test_knn)[:, 1]                         # P(Purchase=1) na teście
pred_test = (proba_test >= 0.5).astype(int)                              # klasy przy progu 0.5

auc_test = roc_auc_score(y_test, proba_test)                             # AUC na teście
acc_test = accuracy_score(y_test, pred_test)                             # accuracy na teście

print("BEST:", best)                                                     # najlepsze k/weights z CV
print(f"TEST: AUC={auc_test:.4f} | ACC={acc_test:.4f}")                  # szybki wynik


BEST: {'auc': 0.8754438775510204, 'k': 15, 'w': 'uniform'}
TEST: AUC=0.9028 | ACC=0.8133


In [36]:
# ocena klasyfikacji na teście
pred_test = knn.predict(X_test_knn)                        # klasy (test)
proba_test = knn.predict_proba(X_test_knn)[:, 1]           # P(Purchase=1) (test)

acc = accuracy_score(y_test, pred_test)                    # (TP+TN)/N na teście
prec = precision_score(y_test, pred_test, zero_division=0) # TP/(TP+FP) na teście
rec = recall_score(y_test, pred_test, zero_division=0)     # TP/(TP+FN) na teście
f1 = f1_score(y_test, pred_test, zero_division=0)          # 2*P*R/(P+R) na teście
auc = roc_auc_score(y_test, proba_test)                    # ROC-AUC na teście
cm = confusion_matrix(y_test, pred_test)                   # [[TN,FP],[FN,TP]]

print(f"TEST METRICS: ACC={acc:.4f} | PREC={prec:.4f} | REC={rec:.4f} | F1={f1:.4f} | AUC={auc:.4f}")
print("CONFUSION MATRIX [[TN,FP],[FN,TP]]:")
print(cm)


TEST METRICS: ACC=0.8133 | PREC=0.8092 | REC=0.8200 | F1=0.8146 | AUC=0.9028
CONFUSION MATRIX [[TN,FP],[FN,TP]]:
[[242  58]
 [ 54 246]]


In [37]:
# Lift chart (TEST) – KNN proba_test; decyle 1..10; lift=rr_decyl/rr_overall_test; etykiety + baseline (y=1.0)
rr_overall_test = float(np.mean(y_test))  # baseline na teście

t = pd.DataFrame({"Purchase": np.asarray(y_test), "proba": np.asarray(proba_test)})  # y + score
t = t.sort_values("proba", ascending=False).reset_index(drop=True)                   # sort malejąco
t["decile"] = (np.floor(np.arange(len(t)) / (len(t) / 10)) + 1).astype(int)          # 1..10
t["decile"] = t["decile"].clip(1, 10)                                                # safety

lift_tbl = (t.groupby("decile")
              .agg(customers=("Purchase", "size"),
                   purchases=("Purchase", "sum"))
              .reset_index())
lift_tbl["rr_decyl"] = lift_tbl["purchases"] / lift_tbl["customers"]                 # rr w decylu
lift_tbl["rr_overall_test"] = rr_overall_test                                        # baseline
lift_tbl["lift"] = lift_tbl["rr_decyl"] / lift_tbl["rr_overall_test"]                # lift
lift_tbl = lift_tbl.sort_values("decile").reset_index(drop=True)                     # 1..10

lift_tbl["txt"] = (                                                                    # etykiety na słupkach
    "n=" + lift_tbl["customers"].astype(int).astype(str) +
    "<br>buy=" + lift_tbl["purchases"].astype(int).astype(str) +
    "<br>rr=" + lift_tbl["rr_decyl"].round(4).astype(str)
)

fig = px.bar(                                                                          # słupki
    lift_tbl,
    x="decile",
    y="lift",
    text="txt",
    hover_data=["customers", "purchases", "rr_decyl", "rr_overall_test"],
    title=f"Lift Chart (TEST) – KNN | rr_overall_test={rr_overall_test:.4f} | baseline lift=1.0",
    template="plotly_dark"
)

fig.update_traces(textposition="outside", cliponaxis=False)                            # tekst nad słupkami
fig.add_hline(y=1.0, annotation_text="baseline (lift=1.0)", annotation_position="top left")  # baseline
fig.update_layout(
    xaxis_title="Decyl (1 = najwyższe P(Purchase=1), 10 = najniższe)",
    yaxis_title="Lift = rr_decyl / rr_overall_test",
    uniformtext_minsize=8,
    uniformtext_mode="hide"
)

fig.show()
lift_tbl


,decile,customers,purchases,rr_decyl,rr_overall_test,lift,txt
0,1,60,58,0.966667,0.5,1.933333,n=60<br>buy=58<br>rr=0.9667
1,2,60,55,0.916667,0.5,1.833333,n=60<br>buy=55<br>rr=0.9167
2,3,60,50,0.833333,0.5,1.666667,n=60<br>buy=50<br>rr=0.8333
3,4,60,47,0.783333,0.5,1.566667,n=60<br>buy=47<br>rr=0.7833
4,5,60,32,0.533333,0.5,1.066667,n=60<br>buy=32<br>rr=0.5333
5,6,60,25,0.416667,0.5,0.833333,n=60<br>buy=25<br>rr=0.4167
6,7,60,26,0.433333,0.5,0.866667,n=60<br>buy=26<br>rr=0.4333
7,8,60,6,0.100000,0.5,0.200000,n=60<br>buy=6<br>rr=0.1
8,9,60,1,0.016667,0.5,0.033333,n=60<br>buy=1<br>rr=0.0167
9,10,60,0,0.000000,0.5,0.000000,n=60<br>buy=0<br>rr=0.0


In [39]:
# Cumulative gains chart (TEST) – sort po proba_test malejąco; X=% populacji; Y=% zakupów złapanych; baseline diagonal

t = pd.DataFrame({"Purchase": np.asarray(y_test), "proba": np.asarray(proba_test)})  # y + score
t = t.sort_values("proba", ascending=False).reset_index(drop=True)                   # sort malejąco

t["cum_pop"] = (np.arange(len(t)) + 1) / len(t)                                      # skumulowany % populacji (0..1)
tot_buy = float(t["Purchase"].sum())                                                 # liczba zakupów w teście
t["cum_buy"] = t["Purchase"].cumsum() / (tot_buy if tot_buy > 0 else 1.0)            # skumulowany % zakupów (0..1)

g = (t.groupby((t["cum_pop"] * 10).astype(int))                                       # progi ~10%,20%,...100%
       .tail(1)[["cum_pop", "cum_buy"]]
       .reset_index(drop=True))
g.loc[len(g)] = [1.0, 1.0]

fig = px.line(
    g,
    x="cum_pop",
    y="cum_buy",
    markers=True,
    title="Cumulative Gains Chart (TEST) – KNN",
    labels={"cum_pop": "Cumulative % of population", "cum_buy": "Cumulative % of purchases captured"},
    template="plotly_dark"
)

fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", name="Random (baseline)")) # diagonala
fig.update_layout(xaxis_tickformat=".0%", yaxis_tickformat=".0%")                    # procenty na osiach
fig.show()

g


,cum_pop,cum_buy
0,0.098333,0.190000
1,0.198333,0.373333
2,0.298333,0.543333
3,0.398333,0.700000
4,0.498333,0.803333
5,0.598333,0.886667
6,0.698333,0.973333
7,0.798333,0.996667
8,0.898333,1.000000
9,0.998333,1.000000
